In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import findspark
findspark.init("/usr/local/spark/spark-2.4.4-bin-hadoop2.7")
import pyspark as spark
from pyspark.sql.functions import udf,col, rand
from pyspark.sql.types import StringType

In [ ]:
spark.SparkConf()

In [ ]:
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [47]:
df = sql.read.option("delimiter",'\t').option("header",'true').csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_reads.gct')
df = df.withColumn('ensg',udf(lambda x: x[:15], StringType())(col('Name')))
df.registerTempTable('gtex')

In [ ]:
genes = sql.read.option("header",'true').csv('/Users/filippo/Developer/tesi/genes.txt')
genes.registerTempTable("genes")
genes.select("type_of_gene").distinct().show()

AnalysisException: 'Path does not exist: file:/Users/filippo/Developer/tesi/genes.txt;'

In [ ]:
pc = sql.sql("SELECT * FROM genes WHERE type_of_gene='protein-coding'")
pc_list = list(pc.select('_c0').orderBy(rand()).toPandas().values.T[0])

In [ ]:
df = df.filter(col("ensg").isin(pc_list))

In [ ]:
df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("mainTable.csv")

In [ ]:
df.toPandas().to_csv("mainTable.csv", index=True, header=True)

## GTEx Biospecimen
[GTEX_biospecimen](https://gtexportal.org/home/samplesPage)

In [37]:
#df_file = sql.createDataFrame(pd.read_csv("https://storage.googleapis.com/gtex_analysis_v7/annotations/GTEx_v7_Annotations_SampleAttributesDS.txt", sep='\t').loc[:,['SAMPID','SMTS', 'SMTSD']])
df_file=sql.read.option('delimiter',',').option("header",'true').csv("files.dat")
df_file.registerTempTable("files")
df_file.describe()

DataFrame[summary: string, sample: string, primary_site: string, secondary_site: string]

In [36]:
#df_sites=pd.DataFrame()
#df_sites.insert(0,'sample',df_file.select('SAMPID').toPandas().values.T[0])
#df_sites.insert(1,'primary_site',df_file.select('SMTS').toPandas().values.T[0])
#df_sites.insert(2,'secondary_site', df_file.select('SMTSD').toPandas().values.T[0])
#df_sites.set_index('sample', inplace=True)
#df_sites.head()
#df_sites.to_csv("files.dat", index=True, header=True)

In [38]:
df_file.select('primary_site').distinct().show()

+---------------+
|   primary_site|
+---------------+
|           Skin|
|  Adrenal Gland|
| Adipose Tissue|
|      Pituitary|
| Salivary Gland|
|         Spleen|
|          Colon|
|          Nerve|
|          Blood|
|         Kidney|
|   Blood Vessel|
|         Muscle|
|       Prostate|
|Small Intestine|
|      Esophagus|
|        Bladder|
|          Liver|
|        Stomach|
|        Thyroid|
|          Heart|
+---------------+
only showing top 20 rows



In [39]:
available_tissues=df_file.groupby('primary_site').count().sort("count", ascending=False).toPandas()
available_tissues

,primary_site,count
0,Blood,2561
1,Brain,2076
2,Skin,1362
3,Esophagus,1111
4,Blood Vessel,1041
5,Adipose Tissue,886
6,Heart,727
7,Muscle,718
8,Lung,607
9,Thyroid,564


In [40]:
#selectedTissues = ['Blood', 'Brain', 'Lung', 'Thyroid', 'Breast', 'Skin', 'Pancreas', 'Testis', 'Adipose Tissue', 'Colon', 'Nerve', 'Heart', 'Kidney', 'Uterus', 'Adrenal Gland']
selectedTissues = available_tissues['primary_site'][:10].values
selectedTissues

array(['Blood', 'Brain', 'Skin', 'Esophagus', 'Blood Vessel',
       'Adipose Tissue', 'Heart', 'Muscle', 'Lung', 'Thyroid'],
      dtype=object)

In [63]:
limit=int(1000/len(selectedTissues))
needed_columns = []
for tissue in selectedTissues:
    needed_columns = np.concatenate((needed_columns,
                                 list(set(sql.sql("SELECT * FROM files WHERE primary_site='%s'"%(tissue))
                                .orderBy(rand())
                                .limit(limit)
                                .select("sample")
                                .toPandas().values.T[0])-{'Name'}-{'Description'})))
needed_columns=needed_columns[np.isin(needed_columns,df.columns)]
np.random.shuffle(needed_columns)
needed_columns = np.concatenate([['ensg'],needed_columns])

In [60]:
needed_columns

array(['ensg', 'GTEX-18D9U-1026-SM-72D5R', 'GTEX-RN64-0526-SM-2TO4V', ...,
       'GTEX-13QJC-0626-SM-5S2O9', 'GTEX-PSDG-1126-SM-2S1ON',
       'GTEX-11OF3-0008-SM-5S2NH'], dtype='<U32')

In [62]:
df.select('GTEX-RN64-0526-SM-2TO4V')

AnalysisException: "cannot resolve '`GTEX-RN64-0526-SM-2TO4V`' given input columns: [GTEX-11GSO-2426-SM-5A5LY, GTEX-13G51-1226-SM-5K7Z3, GTEX-13JVG-0926-SM-5IJE1, GTEX-12WSC-1726-SM-5GCN3, GTEX-13FXS-0011-R10a-SM-5J2MA, GTEX-Q2AI-0008-SM-48U2H, GTEX-YFC4-0011-R11a-SM-4SOK6, GTEX-T2IS-0011-R3A-SM-32QPB, GTEX-13N1W-0326-SM-5IJDJ, GTEX-VUSG-2826-SM-4KKZM, GTEX-17F96-0126-SM-7EWDU, GTEX-12WSJ-1126-SM-5LZVJ, GTEX-14A5I-3026-SM-5TDEC, GTEX-13OVI-2526-SM-5KM4V, GTEX-11TUW-2926-SM-5BC59, GTEX-12696-2726-SM-5FQUL, GTEX-X4EP-0726-SM-3P5YJ, GTEX-16AAH-1326-SM-7KFRG, GTEX-QV44-0326-SM-4R1KD, GTEX-ZTX8-0002-SM-4YCEU, GTEX-145LS-0011-R6b-SM-5PNWL, GTEX-17HGU-0526-SM-793BX, GTEX-QVJO-0011-R5A-SM-2S1QM, GTEX-13O1R-1826-SM-5KM3B, GTEX-1A32A-0726-SM-731D4, GTEX-U412-0008-SM-4DXTE, GTEX-16GPK-1326-SM-7KUE4, GTEX-14BIN-0011-R9a-SM-5SI76, GTEX-18464-0011-R10a-SM-72D6X, GTEX-Y8LW-0008-SM-4V6G5, GTEX-ZPU1-0004-SM-4WWAV, GTEX-14C5O-2326-SM-73KYU, GTEX-PWCY-0526-SM-5P9HG, GTEX-16AAH-1626-SM-7EWE2, GTEX-11LCK-1

In [44]:
df.select(list(needed_columns)).repartition(1).format("com.databricks.spark.csv").option("header", "true").save("mainTable_all.csv", header=True, index=False)

AnalysisException: "cannot resolve '`GTEX-U3ZN-0001-SM-3G3TS`' given input columns: [GTEX-14PN4-0426-SM-6AJ9N, GTEX-13FTW-1226-SM-5LZZ1, GTEX-1314G-2026-SM-5LZUM, GTEX-WOFL-1226-SM-4OOST, GTEX-1A3MX-0011-R2b-SM-7EWFL, GTEX-13CF2-1826-SM-5N9GK, GTEX-14B4R-1426-SM-5Q5CG, GTEX-ZF2S-0826-SM-4WWBI, GTEX-ZVZQ-0008-SM-51MSK, GTEX-X4EP-0326-SM-3P5Z6, GTEX-PWCY-1326-SM-48TCU, GTEX-WFON-1126-SM-4LVMA, GTEX-PX3G-2026-SM-48U1H, GTEX-XBED-0006-SM-47JXO, GTEX-18D9U-0826-SM-7LG5K, GTEX-1CAMS-1426-SM-7IGPM, GTEX-13FTW-0426-SM-5K7X2, GTEX-18464-0011-R9a-SM-731BK, GTEX-XGQ4-0826-SM-4AT4T, GTEX-117YX-1926-SM-5GIEC, GTEX-13OVG-1926-SM-5L3DK, GTEX-13FTW-0626-SM-5IFEX, GTEX-WVJS-0226-SM-4MVOU, GTEX-131XH-0426-SM-5DUWU, GTEX-XPT6-0226-SM-4B65L, GTEX-11ZTS-1126-SM-5LU9X, GTEX-1399T-1226-SM-5P9J5, GTEX-15SHV-0326-SM-6M475, GTEX-XMD3-0008-SM-4AT4V, GTEX-ZLFU-0626-SM-4WWBO, GTEX-WHWD-1826-SM-3LK6I, GTEX-Z93S-0926-SM-5HL8B, GTEX-YEC4-0826-SM-5P9FV, GTEX-13FHO-2826-SM-5K7ZA, GTEX-X585-0011-R5A-SM-46MVI, GTEX-12WSD

In [ ]:
df_file.filter(col('_c0').isin(list(needed_columns))).toPandas().to_csv("files_tissues.dat", header=True, index=False)

# U

In [ ]:
tissue = 'Breast'

In [ ]:
sample_list = df_sites[df_sites['primary_site']==tissue].index.values
subdf = df.loc[:,sample_list]

In [ ]:
odf = pd.read_csv("meanVariances.csv", header=0, index_col=0)
odf.insert(3,"A",pd.read_csv("A.dat", header=None).values)
odf.head()

In [ ]:
O = odf[odf.index.isin(pc.index)]['occurrence']
O_nc = odf[~odf.index.isin(pc.index)]['occurrence']

In [ ]:
bins = 20
rang = (0-0.5/20,1+0.5/20)
fig = plt.figure()
plt.hist(O[O<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='coding')
plt.hist(O_nc[O_nc<=1], histtype='step', lw=4, density=True, bins=bins, range=rang, label='non-coding')
#plt.title(tissue, fontsize=18)
plt.xlabel('$O_i$', fontsize=16)
plt.ylabel('pdf', fontsize=16)
plt.legend(loc='upper left', fontsize=16)
plt.show()
fig.savefig("U_gtex_cnc.pdf")

In [ ]:
A = odf['A'].values
A_c = odf[odf.index.isin(pc.index)]['A'].values
A_nc = odf[~odf.index.isin(pc.index)]['A'].values
fig=plt.figure()
plt.plot(np.arange(0,len(A)), 1./np.arange(0,len(A)), ls='--', c='g', label='$r^-1$')
#plt.plot(np.arange(0,len(A)), np.sort(A)[::-1]/np.sum(A))
#plt.plot(np.arange(0,len(A_c)), np.sort(A_c)[::-1]/np.sum(A), label='coding genes')
plt.plot(np.arange(0,len(A_nc)), np.sort(A_nc)[::-1]/np.sum(A), label='non coding genes')
plt.xscale('log')
plt.yscale('log')
plt.ylabel('$f_i$', fontsize=18)
plt.xlabel("Rank_i", fontsize=18)
plt.legend(fontsize=18)
fig.savefig("globalZipf_nc.pdf")